### Method 1 : Using `Pipeline`

In [1]:
from transformers import pipeline

ppl = pipeline(task="question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")

question = "What is Jersey Act ?"
context = '''
The Jersey Act was introduced to prevent the registration of most American-bred Thoroughbred horses in the British General Stud Book.
It had its roots in the desire of British horse breeders to halt the influx of American-bred racehorses of possibly impure bloodlines
 during the early 20th century. Many American-bred horses were exported to Europe to race and retire to a breeding career after a number of U.S.
  states banned gambling, which depressed Thoroughbred racing as well as breeding in the United States. The loss of breeding records during
  the American Civil War and the late beginning of the registration of American Thoroughbreds led many in the British racing establishment
   to doubt that the American-bred horses were purebred.
'''

result = ppl(question = question, context=context)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cpu


In [2]:
result

{'score': 0.03040647879242897,
 'start': 31,
 'end': 100,
 'answer': 'to prevent the registration of most American-bred Thoroughbred horses'}

### Method 2 : Using `AutoModelForQuestionAnswering` and `AutoTokenizer`

In [3]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import torch

In [4]:
ckpt = "bert-large-uncased-whole-word-masking-finetuned-squad"
model = AutoModelForQuestionAnswering.from_pretrained(ckpt)
tokenizer = AutoTokenizer.from_pretrained(ckpt)

question = "What is Jersey Act ?"
context = '''
The Jersey Act was introduced to prevent the registration of most American-bred Thoroughbred horses in the British General Stud Book. It had its roots in the desire of British horse breeders to halt the influx of American-bred racehorses of possibly impure bloodlines during the early 20th century. Many American-bred horses were exported to Europe to race and retire to a breeding career after a number of U.S. states banned gambling, which depressed Thoroughbred racing as well as breeding in the United States. The loss of breeding records during the American Civil War and the late beginning of the registration of American Thoroughbreds led many in the British racing establishment to doubt that the American-bred horses were purebred.
'''

inputs = tokenizer(question, context, return_tensors='pt')

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
with torch.no_grad():
    output = model(**inputs)

In [6]:
output

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-5.9799, -5.2326, -7.3812, -7.0938, -8.5354, -9.6828, -5.9798,  1.8995,
         -0.0590, -3.9374, -0.2013,  0.4523,  2.8736,  1.9361, -4.2128, -2.7480,
         -6.0425, -4.3723, -3.2908, -6.8249, -5.2264, -3.4335, -4.4484, -4.2624,
         -3.3569, -1.4981, -3.2093, -3.4222, -3.7778, -5.9799,  0.8526, -3.5236,
         -4.1442, -3.1097, -5.0712, -2.4119, -1.0609, -6.5987, -0.8546, -3.0012,
         -4.7014, -1.0428, -1.5233, -5.1667, -3.7215, -6.0956, -3.3341, -7.4709,
         -5.5022, -4.6659, -5.8692, -5.9974, -2.5143, -4.0405, -5.4671, -4.9356,
         -4.9968, -6.2823, -6.6198, -5.3579, -5.4446, -6.3360, -6.0737, -3.6126,
         -4.6887, -8.1716, -6.6218, -5.7185, -7.2662, -5.4052, -7.9011, -5.7640,
         -7.0935, -6.0692, -8.7138, -6.2634, -7.6737, -7.5064, -5.3030, -6.9123,
         -6.2390, -6.6331, -6.7438, -8.1749, -5.6635, -8.0700, -8.0389, -8.7232,
         -7.0600, -4.6931, -5.7083, -7.5998, -6.5596, -4

In [7]:
start_index = output["start_logits"].argmax()
end_index = output["end_logits"].argmax()

In [8]:
start_index

tensor(12)

In [9]:
end_index

tensor(28)

In [10]:
output_ids = inputs["input_ids"][0][start_index:end_index+1]
" ".join(tokenizer.convert_ids_to_tokens(output_ids))

'to prevent the registration of most american - bred thoroughbred horses in the british general stud book'

In [11]:
inputs

{'input_ids': tensor([[  101,  2054,  2003,  3933,  2552,  1029,   102,  1996,  3933,  2552,
          2001,  3107,  2000,  4652,  1996,  8819,  1997,  2087,  2137,  1011,
         13680, 18359,  5194,  1999,  1996,  2329,  2236, 16054,  2338,  1012,
          2009,  2018,  2049,  6147,  1999,  1996,  4792,  1997,  2329,  3586,
         20823,  2000,  9190,  1996, 18050,  1997,  2137,  1011, 13680, 25068,
          2015,  1997,  4298, 17727,  5397,  2668, 12735,  2076,  1996,  2220,
          3983,  2301,  1012,  2116,  2137,  1011, 13680,  5194,  2020, 15612,
          2000,  2885,  2000,  2679,  1998, 11036,  2000,  1037,  8119,  2476,
          2044,  1037,  2193,  1997,  1057,  1012,  1055,  1012,  2163,  7917,
         12219,  1010,  2029, 14777, 18359,  3868,  2004,  2092,  2004,  8119,
          1999,  1996,  2142,  2163,  1012,  1996,  3279,  1997,  8119,  2636,
          2076,  1996,  2137,  2942,  2162,  1998,  1996,  2397,  2927,  1997,
          1996,  8819,  1997,  2137, 1

In [12]:
inputs["input_ids"][0]

tensor([  101,  2054,  2003,  3933,  2552,  1029,   102,  1996,  3933,  2552,
         2001,  3107,  2000,  4652,  1996,  8819,  1997,  2087,  2137,  1011,
        13680, 18359,  5194,  1999,  1996,  2329,  2236, 16054,  2338,  1012,
         2009,  2018,  2049,  6147,  1999,  1996,  4792,  1997,  2329,  3586,
        20823,  2000,  9190,  1996, 18050,  1997,  2137,  1011, 13680, 25068,
         2015,  1997,  4298, 17727,  5397,  2668, 12735,  2076,  1996,  2220,
         3983,  2301,  1012,  2116,  2137,  1011, 13680,  5194,  2020, 15612,
         2000,  2885,  2000,  2679,  1998, 11036,  2000,  1037,  8119,  2476,
         2044,  1037,  2193,  1997,  1057,  1012,  1055,  1012,  2163,  7917,
        12219,  1010,  2029, 14777, 18359,  3868,  2004,  2092,  2004,  8119,
         1999,  1996,  2142,  2163,  1012,  1996,  3279,  1997,  8119,  2636,
         2076,  1996,  2137,  2942,  2162,  1998,  1996,  2397,  2927,  1997,
         1996,  8819,  1997,  2137, 18359,  2015,  2419,  2116, 

In [13]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

['[CLS]',
 'what',
 'is',
 'jersey',
 'act',
 '?',
 '[SEP]',
 'the',
 'jersey',
 'act',
 'was',
 'introduced',
 'to',
 'prevent',
 'the',
 'registration',
 'of',
 'most',
 'american',
 '-',
 'bred',
 'thoroughbred',
 'horses',
 'in',
 'the',
 'british',
 'general',
 'stud',
 'book',
 '.',
 'it',
 'had',
 'its',
 'roots',
 'in',
 'the',
 'desire',
 'of',
 'british',
 'horse',
 'breeders',
 'to',
 'halt',
 'the',
 'influx',
 'of',
 'american',
 '-',
 'bred',
 'racehorse',
 '##s',
 'of',
 'possibly',
 'imp',
 '##ure',
 'blood',
 '##lines',
 'during',
 'the',
 'early',
 '20th',
 'century',
 '.',
 'many',
 'american',
 '-',
 'bred',
 'horses',
 'were',
 'exported',
 'to',
 'europe',
 'to',
 'race',
 'and',
 'retire',
 'to',
 'a',
 'breeding',
 'career',
 'after',
 'a',
 'number',
 'of',
 'u',
 '.',
 's',
 '.',
 'states',
 'banned',
 'gambling',
 ',',
 'which',
 'depressed',
 'thoroughbred',
 'racing',
 'as',
 'well',
 'as',
 'breeding',
 'in',
 'the',
 'united',
 'states',
 '.',
 'the',
 'l